In [1]:
import os
import time
import numpy as np
#import matplotlib.pyplot as plt
from collections import deque
import gym
import pandas as pd
from pathlib import Path
import sys
import random
#from joblib import dump,load
import datetime
import tensorflow as tf
#import pybullet_envs

from environments import PortfolioEnvironment
from utils import ReplayBuffer, ReplayBufferMultiObs
from policies import TD3,TD3p

In [2]:
def evaluate_policy(policy, eval_episodes=10):
    avg_reward = 0.0
    for _ in range(eval_episodes):
        state = env.reset()
        done = False
        while not done:
            action = policy.select_action(np.array(state))
            #print(action)
            #action = tf.nn.softmax(action+ np.random.normal(0,1,size=4)).numpy()
            state, reward, done, _ = env.step(action)
            #print(_)
            avg_reward += reward
            #print(action)
    avg_reward /= eval_episodes
    print(f'Average Reward: {avg_reward}')
    return avg_reward

In [3]:
seed = 0
start_timesteps = 1e4 # Number of timesteps in which the model choose a random action, after that number starts using the policy
eval_freq = 5e3 # policy evaluation frequency in timesteps
max_timesteps = 100#5e5 
save_models = True 
expl_noise = 0.3 # Exploration noise 
batch_size = 100 
discount = 0.99 # reward Discount factor gamma 
tau = 0.005 # target weights update ratio
policy_noise = 0.2 # std deviation of gaussian noise to be added to the action, for exploration purposes
noise_clip = 0.5 # max value of gaussian noise added to action
policy_freq = 2 # actor model weights update frecuency

In [51]:
aapl = pd.read_csv('./data/AAPL.csv')[['close','high','low']]
xom = pd.read_csv('./data/XOM.csv')[['close','high','low']]
tsla = pd.read_csv('./data/TSLA.csv')[['close','high','low']]
tickers = ['AAPL','XOM','TSLA','TSLA','TSLA','TSLA','TSLA','TSLA','TSLA','TSLA','TSLA']
assets_data_list = [aapl,xom,tsla,tsla,tsla,tsla,tsla,tsla,tsla,tsla,tsla]

In [5]:
#xom_daily = pd.read_csv('BTC-USD.csv')[['Adj Close','High','Low']]
#tsla_daily = pd.read_csv('DOGE-USD.csv')[['Adj Close','High','Low']]
xom_daily = pd.read_csv('./data/XOM_y.csv')[['Adj Close','High','Low']]
tsla_daily = pd.read_csv('./data/TSLA_y.csv')[['Adj Close','High','Low']]
aapl_daily = pd.read_csv('./data/AAPL_y.csv')[['Adj Close','High','Low']]
tickers_daily = ['AAPL','XOM','TSLA']
assets_data_list_daily = [aapl_daily,xom_daily,tsla_daily]

In [56]:
#env = PortfolioEnvironment(tickers_daily,assets_data_list_daily,fee=0.025,look_back_window=12,max_steps=24)
env = PortfolioEnvironment(tickers,assets_data_list,fee=0.0025,look_back_window=50,max_steps=200)
#env = gym.wrappers.FlattenObservation(env)
#env = gym.make("HalfCheetahBulletEnv-v0")

In [10]:
#state_dim = env.observation_space.shape[0]
state_dim = env.observation_space["data"].shape
action_dim = env.action_space.shape[0]
max_action = float(env.action_space.high[0])

TypeError: 'NoneType' object is not subscriptable

In [8]:
policy = TD3p(state_dim,action_dim,max_action)

replay_buffer = ReplayBuffer()
evaluations = [evaluate_policy(policy,1)]

[0.18763536 0.17802846 0.23229195 0.22262447 0.17941977]
Average Reward: 403.7224399505212


In [21]:
total_timesteps = 0
timesteps_since_eval = 0
episode_num = 0
done = True

In [22]:
start_time=time.time()
info_list = []
while total_timesteps < max_timesteps:
    if done:
        if total_timesteps != 0:
            print(f'Total timesteps: {total_timesteps} Episode Num: {episode_num} Reward: {episode_reward} Portfolio Value:{env.portfolio_value_units}')
            #policy.train(replay_buffer,episode_timesteps, batch_size, discount, tau, policy_noise, noise_clip, policy_freq)
        
        if timesteps_since_eval >= eval_freq:
            timesteps_since_eval %= eval_freq
            evaluations.append(evaluate_policy(policy))
            #policy.save(file_name)
            #np.save(f'policy_evaluation',evaluations)
        
        obs = env.reset()
        done = False
        
        episode_reward = 0
        episode_timesteps = 0
        episode_num +=1
        
    if total_timesteps < start_timesteps:
        action = tf.nn.softmax(env.action_space.sample()+ np.random.normal(0,1,size=env.action_space.shape[0])).numpy()
    else:
        action = policy.select_action(np.array(obs))
        if expl_noise != 0: 
            #action = (action + np.random.normal(0,expl_noise,size=env.action_space.shape[0])).clip(env.action_space.low, env.action_space.high)
            action = tf.nn.softmax((action + np.random.normal(0,expl_noise,size=env.action_space.shape[0]))).numpy()
    
    #print(action)
    next_obs,reward,done, info = env.step(action)
    info_list.append(info)
    #done_bool = 0 if episode_timesteps +1 == env._max_episode_steps else float(done)
    done_bool = 0 if episode_timesteps +1 == env.max_steps else float(done)
    
    #print(reward)
    episode_reward += reward
    replay_buffer.add((obs,next_obs,action,reward,done_bool))
    
    obs = next_obs
    episode_timesteps +=1
    total_timesteps +=1
    timesteps_since_eval +=1

print(evaluations.append(evaluate_policy(policy)))
#policy.save()
#np.save(f'policy_evaluation',evaluations)
print(f'Tiempo de entrenamiento: {time.time()-start_time} segundos')

[0.06303699 0.1140858  0.52352004 0.29935717]
Total timesteps: 201 Episode Num: 1 Reward: -27453.365945989077 Portfolio Value:72546.63405401092
[0.1243304  0.02700867 0.27515057 0.57351036]
Total timesteps: 402 Episode Num: 2 Reward: -26801.481608005255 Portfolio Value:73198.51839199474
[0.13870454 0.05691498 0.07375519 0.73062529]
Total timesteps: 603 Episode Num: 3 Reward: -27614.58823299875 Portfolio Value:72385.41176700125
[0.03002339 0.00870804 0.34769574 0.61357283]
Total timesteps: 804 Episode Num: 4 Reward: -27642.123693563364 Portfolio Value:72357.87630643664
[0.01531772 0.17686942 0.72741775 0.08039512]
Total timesteps: 1005 Episode Num: 5 Reward: -27377.329413282394 Portfolio Value:72622.6705867176
[0.52106201 0.19097793 0.25261252 0.03534754]
Total timesteps: 1206 Episode Num: 6 Reward: -26906.15772813534 Portfolio Value:73093.84227186466
[0.00497202 0.79702448 0.16314928 0.03485423]
Total timesteps: 1407 Episode Num: 7 Reward: -26935.979703103614 Portfolio Value:73064.0202

NameError: name 'os' is not defined

In [172]:
data = tf.expand_dims(np.random.normal(0,1,size=(11,50,3)),axis=0)
def expandDims(x):
    expX = tf.expand_dims(x, axis=1)
    expX = tf.expand_dims(expX, axis=1)
    expX = tf.expand_dims(expX, axis=0)
    return expX


weights = np.ones((12))
bias = weights[0]
weights = weights[1:]
#data = tf.expand_dims(np.array([data[:,:,i] for i in range(data.shape[2])]),axis=0)
print(data.shape)
#data = np.array([[data[:,:,i] for i in range(data.shape[2])]])
network = tf.keras.layers.Conv2D(filters=2, kernel_size=(1,3), activation='relu')(data)
network = tf.keras.layers.Conv2D(filters=20, kernel_size=(1,48), activation='relu')(network)
weights = tf.keras.layers.Lambda(expandDims)(weights)
network = tf.concat([network,weights],3)
network = tf.keras.layers.Conv2D(filters=1, kernel_size=(1,1))(network)
bias = tf.keras.layers.Lambda(expandDims)([bias])
network = tf.concat([bias,network],1)
network = tf.keras.layers.Flatten()(network)
tf.keras.layers.Activation('softmax')(network)
#tf.nn.softmax(network,axis=1)
#network

(1, 11, 50, 3)


<tf.Tensor: shape=(1, 12), dtype=float32, numpy=
array([[0.1124526 , 0.07600761, 0.06547023, 0.06233483, 0.07057125,
        0.0594045 , 0.11878745, 0.0843537 , 0.10976241, 0.07272332,
        0.09194978, 0.07618225]], dtype=float32)>

In [61]:
test_buffer = ReplayBufferMultiObs()
obs = env.reset()
action = env.action_space.sample()
reward = 100
done_bool = 1

for i in range(100):
    test_buffer.add((obs,obs,action,reward,done_bool))
    
obs = test_buffer.sample(100)[0]

In [22]:
def expandDims(x):
    expX = tf.expand_dims(x, axis=1)
    expX = tf.expand_dims(expX, axis=1)
    expX = tf.expand_dims(expX, axis=0)
    return expX

In [69]:
def expand_dims(x):
    expX = tf.expand_dims(x, axis=-1)
    expX = tf.expand_dims(expX, axis=-1)
    return expX

In [67]:
#data = tf.expand_dims(np.random.normal(0,1,size=(11,50,3)),axis=0)

bias = obs["weights"][:,0].reshape(-1,1)
weights = obs["weights"][:,1:]
#data = tf.expand_dims(np.array([data[:,:,i] for i in range(data.shape[2])]),axis=0)
#print(data.shape)
#data = np.array([[data[:,:,i] for i in range(data.shape[2])]])
network = tf.keras.layers.Conv2D(filters=2, kernel_size=(1,3), activation='relu')(obs["data"])
network = tf.keras.layers.Conv2D(filters=20, kernel_size=(1,48), activation='relu')(network)
weights = tf.keras.layers.Lambda(expandDims)(weights)
network = tf.concat([network,weights],3)
network = tf.keras.layers.Conv2D(filters=1, kernel_size=(1,1))(network)
bias = tf.keras.layers.Lambda(expandDims)(bias)
network = tf.concat([bias,network],1)
network = tf.keras.layers.Flatten()(network)
tf.keras.layers.Activation('softmax')(network)

<tf.Tensor: shape=(100, 12), dtype=float32, numpy=
array([[0.16221087, 0.07616808, 0.07607599, ..., 0.07617167, 0.07617167,
        0.07617167],
       [0.16221087, 0.07616808, 0.07607599, ..., 0.07617167, 0.07617167,
        0.07617167],
       [0.16221087, 0.07616808, 0.07607599, ..., 0.07617167, 0.07617167,
        0.07617167],
       ...,
       [0.16221087, 0.07616808, 0.07607599, ..., 0.07617167, 0.07617167,
        0.07617167],
       [0.16221087, 0.07616808, 0.07607599, ..., 0.07617167, 0.07617167,
        0.07617167],
       [0.16221087, 0.07616808, 0.07607599, ..., 0.07617167, 0.07617167,
        0.07617167]], dtype=float32)>

In [75]:
_obs = obs["data"]
_bias = tf.keras.layers.Lambda(expand_dims)(obs["weights"][:,0].reshape(-1,1))
_weights = tf.keras.layers.Lambda(expand_dims)(obs["weights"][:,1:])
x1 = tf.keras.layers.Conv2D(filters=2, kernel_size=(1,3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001))(_obs)
x1 = tf.keras.layers.Conv2D(filters=20, kernel_size=(1,48), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001))(x1)
x1 = tf.concat([x1,_weights],3)
x1 = tf.keras.layers.Conv2D(filters=1, kernel_size=(1,1), kernel_regularizer=tf.keras.regularizers.l2(0.001))(x1)
x1 = tf.concat([_bias,x1],1)
x1 = tf.keras.layers.Flatten()(x1)
x1 = tf.keras.layers.Dropout(0.2)(x1)
x1 = tf.keras.layers.Dense(11, activation='relu')(x1)
x1 = tf.concat([actions,x1],1)
x1 = tf.keras.layers.Dense(1, activation='relu')(x1)

In [72]:
actions = test_buffer.sample(100)[2]
actions

array([[0.822 , 0.3184, 0.0932, ..., 0.4487, 0.1805, 0.4817],
       [0.822 , 0.3184, 0.0932, ..., 0.4487, 0.1805, 0.4817],
       [0.822 , 0.3184, 0.0932, ..., 0.4487, 0.1805, 0.4817],
       ...,
       [0.822 , 0.3184, 0.0932, ..., 0.4487, 0.1805, 0.4817],
       [0.822 , 0.3184, 0.0932, ..., 0.4487, 0.1805, 0.4817],
       [0.822 , 0.3184, 0.0932, ..., 0.4487, 0.1805, 0.4817]],
      dtype=float16)

In [74]:
actions.shape

(100, 12)